## 欠損データへの対処

In [1]:
import pandas as pd
from io import StringIO
csv_data = '''A, B, C, D
              1.0 , 2.0, 3.0, 4.0
              5.0 , 6.0,, 8.0
              10.0, 11.0, 12.0,'''
df = pd.read_csv(StringIO(csv_data), skipinitialspace=True)
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


### ・取り除く

In [2]:
# 欠損値をカウント
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [3]:
# 欠損値を含んでいる行を削除
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
# 欠損値を含んでいる列を削除
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
# すべての列がNaNである行だけを削除
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [6]:
# 非NaNの値が4つ未満の行を削除
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [7]:
# 特定の列にNaNがある行を削除
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### ・補完する
**strategyの例**  
'mean'：平均値  
'median'：中央値  
'most_frequent'：最頻値

In [8]:
from sklearn.impute import SimpleImputer
import numpy as np
# NaNを平均値で埋める
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
# データに適合させる
imr = imr.fit(df.values) # データフレームの値をNumPy配列に変換
# 補完を実行
imputed_data = imr.transform(df.values)
imputed_data

# df.fillna(df.mean())  # 平均値でNaNを埋める方法

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

### ・scikit-learnの推定器API
**・scikit-learnのデータ変換APIを使う**  
est.fit(X_train)  
→est.trainsform(X_train), est.trainsform(X_test)  
**・分類器などの予測モデルにscikit-learnのAPIを使う**  
est.fit(X_train, y_train)  
→est.predict(X_test)

---

## カテゴリデータの処理

### ・名義特徴量と順序特徴量

In [9]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2'],
])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [10]:
# 順序特徴量のマッピング
size_mapping = {'XL': 3, 'L': 2, 'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [11]:
# クラスラベルのエンコーディング
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [12]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [13]:
# 元の表現に戻す
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


#### 便利なクラス

In [14]:
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーディング
class_le = LabelEncoder()
# クラスラベルから整数に変換
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [15]:
# クラスラベルを元の文字列に戻す
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

#### one-hotエンコーディング
順序関係のないカテゴリを分けて扱える。  
**その他エンコーディング方式**
- バイナリーエンコーディング(binary encoding)
- カウントエンコーディング(count encoding)

In [16]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
# One-hotエンコーディング
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:, 0].reshape(-1, 1)).toarray()  # 色の列のみをOne-hotエンコード

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [17]:
# 選択的に変換したい場合
from sklearn.compose import ColumnTransformer
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('color_ohe', OneHotEncoder(), [0]),  # 色の列をOne-hotエンコード
    ('nothing', 'passthrough', [1, 2]) # remainder='passthrough'  # 他の列はそのまま
])
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [18]:
# 文字列値を持つ列だけが変換される
pd.get_dummies(df[['color', 'size', 'price']])
pd.get_dummies(df[['color', 'size', 'price']], drop_first=True)  # 最初の列をドロップしてダミー変数の落とし穴を回避

,size,price,color_green,color_red
0,1,10.1,True,False
1,2,13.5,False,True
2,3,15.3,False,False


In [19]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
c_transf = ColumnTransformer([
    ('color_ohe', OneHotEncoder(), [0]),  # 色の列をOne-hotエンコード
    ('nothing', 'passthrough', [1, 2]) # remainder='passthrough'  # 他の列はそのまま
])
c_transf.fit_transform(X).astype(float)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

一つの列が他の列の線形結合で表せる場合、多重共線性が発生しており、線形モデル（回帰やロジスティック回帰など）で問題。  
それを回避している。